In [1]:
import os.path
import time
import traceback

import pandas as pd
import numpy as np

import folium
import folium.plugins
import fuzzywuzzy.process
import geopandas as gpd
import overpy
from tqdm import tqdm

import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 100)

# TODO Update to download all results initially.

/opt/conda/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Load existing steel plant data and perform basic sanity checks:

In [2]:
STEEL_CSV = '../../../../csv_data/steel/steel_dataset_v2-update.csv'
CHECKPOINT_FILE = '../../../../csv_data/steel/steel_dataset_v2-update-with-polygons_checkpoint.pickle'
OUTPUT_FILE_GERMANY = '../../../../csv_data/steel/steel_dataset_v2-update-with-polygons_germany.csv'
OUTPUT_FILE = '../../../../csv_data/steel/steel_dataset_v2-update-with-polygons.csv'
df_steel = pd.read_csv(STEEL_CSV)

df_steel = gpd.GeoDataFrame(df_steel, geometry=gpd.points_from_xy(df_steel['longitude'], df_steel['latitude']))
df_steel.head()

,country,location_id,company_name,number_plants,number_blast_furnace,total_capacity,location_name,latitude,longitude,accuracy,uid,geometry
0,GERMANY,100021,THYSSENKRUPP STEEL EUROPE AG,2,2.0,3700.0,DUISBURG-HAMBORN,51.485334,6.731369,Exact,DEU0001,POINT (6.73137 51.48533)
1,GERMANY,100022,THYSSENKRUPP STEEL EUROPE AG,16,0.0,27544.0,DUISBURG-BRUCKHAUSEN,51.493014,6.740669,Exact,DEU0002,POINT (6.74067 51.49301)
2,GERMANY,100023,THYSSENKRUPP STEEL EUROPE AG,6,2.0,22815.0,DUISBURG-SCHWELGERN,51.506862,6.739497,Exact,DEU0003,POINT (6.73950 51.50686)
3,GERMANY,100024,THYSSENKRUPP STEEL EUROPE AG,16,0.0,29280.0,DUISBURG-BEECKERWERTH,51.483780,6.706625,Exact,DEU0004,POINT (6.70662 51.48378)
4,GERMANY,100025,THYSSENKRUPP STEEL EUROPE AG,3,0.0,1620.0,DUISBURG-HÌÏTTENHEIM,51.365948,6.710592,Exact,DEU0005,POINT (6.71059 51.36595)


In [3]:
print('Number of records with identical lat lon fields: {}'.format(sum(df_steel['latitude'] == df_steel['longitude'])))

Number of records with identical lat lon fields: 1


In [4]:
# plot steel plants on a leaflet map
m = folium.Map()
m.add_child(folium.plugins.FastMarkerCluster(
    df_steel.geometry.apply(lambda g: [g.centroid.y, g.centroid.x]) # plot just the centre points
))
m

Judging from the map, one of the records has implausible coordinates:

In [5]:
df_steel['latitude'].sort_values()
df_steel.iloc[1646]

country                                        UNITED STATES
location_id                                          7900191
company_name                      NORTH STAR BLUESCOPE STEEL
number_plants                                              4
number_blast_furnace                                       0
total_capacity                                          7640
location_name                                      DELTA, OH
latitude                                            -84.0484
longitude                                            41.5685
accuracy                                               Exact
uid                                                  USA0022
geometry                POINT (41.568502 -84.04835200000001)
Name: 1646, dtype: object

We seek to obtain polygon information for each of the steel plants, based on their individual lat/lon coordinates and using OSM data. To this end, using Overpass we will query OSM data for land labelled as 'industrial'. We will illustrate the behaviour of our proposed heuristic using German plant data.

In [6]:
# Limit ourselves to German plants for now...
df_steel_germany = df_steel[df_steel['country'] == 'GERMANY'].copy(deep=True)
print('Number of steel plants in annotations: {}'.format(len(df_steel_germany)))

Number of steel plants in annotations: 92


In [7]:
df_steel_germany.head(20)

,country,location_id,company_name,number_plants,number_blast_furnace,total_capacity,location_name,latitude,longitude,accuracy,uid,geometry
0,GERMANY,100021,THYSSENKRUPP STEEL EUROPE AG,2,2.0,3700.0,DUISBURG-HAMBORN,51.485334,6.731369,Exact,DEU0001,POINT (6.73137 51.48533)
1,GERMANY,100022,THYSSENKRUPP STEEL EUROPE AG,16,0.0,27544.0,DUISBURG-BRUCKHAUSEN,51.493014,6.740669,Exact,DEU0002,POINT (6.74067 51.49301)
2,GERMANY,100023,THYSSENKRUPP STEEL EUROPE AG,6,2.0,22815.0,DUISBURG-SCHWELGERN,51.506862,6.739497,Exact,DEU0003,POINT (6.73950 51.50686)
3,GERMANY,100024,THYSSENKRUPP STEEL EUROPE AG,16,0.0,29280.0,DUISBURG-BEECKERWERTH,51.483780,6.706625,Exact,DEU0004,POINT (6.70662 51.48378)
4,GERMANY,100025,THYSSENKRUPP STEEL EUROPE AG,3,0.0,1620.0,DUISBURG-HÌÏTTENHEIM,51.365948,6.710592,Exact,DEU0005,POINT (6.71059 51.36595)
5,GERMANY,100031,THYSSENKRUPP STEEL EUROPE AG,8,0.0,10460.0,BOCHUM,51.476514,7.175024,Exact,DEU0006,POINT (7.17502 51.47651)
6,GERMANY,100032,THYSSENKRUPP STEEL EUROPE AG,8,0.0,7450.0,DORTMUND,51.538297,7.501288,Exact,DEU0007,POINT (7.50129 51.53830)
7,GERMANY,100034,THYSSENKRUPP STEEL EUROPE AG,2,0.0,665.0,KREUZTAL-EICHEN,50.970501,7.977616,Exact,DEU0008,POINT (7.97762 50.97050)
8,GERMANY,100035,THYSSENKRUPP STEEL EUROPE AG,3,0.0,1422.0,KREUZTAL-FERNDORF,51.168455,7.968379,Exact,DEU0009,POINT (7.96838 51.16846)
9,GERMANY,100037,THYSSENKRUPP STEEL EUROPE AG,10,0.0,2970.0,"ELEKTROBAND, BOCHUM",51.489378,7.249118,Exact,DEU0010,POINT (7.24912 51.48938)


Our proposed heuristic retrieves (multi-)polygon data based on the following order of precedence:

1. If a single Relation exists for the specified lat/lon coordinates with land use designated as Industrial, return all outer Ways associated with the Relation
2. If a single Way exists for the specified lat/lon coordinates with land use designated as Industrial, return the Way
3. If multiple Relations exist for the specified lat/lon coordinates with land use designated as Industrial, perform approximate string matching between the plant name recorded in our data and the names assigned to the Relations. Return all outer Ways associated with the highest-scoring Relation, providing its matching score exceeds a specified threshold and providing the name associated with the Relation is not None.
4. If multiple Ways exist for the specified lat/lon coordinates with land use designated as Industrial, perform approximate string matching between the plant name recorded in our data and the names assigned to the Ways. Return the highest-scoring Way, providing its matching score exceeds a specified threshold and providing the name associated with the Way is not None.

We apply the above heuristic first using a membership-based query 'overpass_area_membership_query', based on the function is_in(). As observed using test data, this query does not always return results. Therefore, as a fall-back method we use a proximity-based query, where we iteratively increase the proximity radius until we obtain one or more candidate Relations or one or more candidate Ways.

Note that a key assumption in rule 1. is that the surrounding Relation is indeed a steel plant. Note that this assumption might be violated when the steel plant is part of a larger industrial area unrelated to steel production. On the other hand, steel plants identifiable as such might be sufficiently large to be unlikely to be part of any larger industrial area. Also, it is worth noting that 1. accounts for the case where a steel plant comprises disparate sites.

Key assumptions in rules 3. and 4. are that name properties are abundant and can be reliably matched to recorded plant names. Note that for the case where multiple potential matches exist, by using name matching and a matching threshold, we operate conservatively.

All rules assume that lat/lon data are precise and that they refer to actual steel plants. If a single potential match (Relation or Way) exists within the specified proximity of the lat/lon value, the match will be assigned regardless of any name matching.

In [8]:
overpass_area_membership_query = """
[out:json][timeout:25];
is_in(LAT, LON)->.a;
(
 way(pivot.a)['landuse'='industrial'];
 relation(pivot.a)['landuse'='industrial'];
 way(pivot.a)[~"name"~"钢"];
 relation(pivot.a)[~"name"~"钢"];
);
out body;
>;
out skel qt;
"""

overpass_proximity_query = """
[out:json][timeout:25];
(
way[landuse='industrial'](around:RADIUS, LAT, LON);
relation[landuse='industrial'](around:RADIUS, LAT, LON);
way[~"name"~"钢"](around:RADIUS, LAT, LON);
relation[~"name"~"钢"](around:RADIUS, LAT, LON);
);
out body;
>;
out skel qt;
"""

overpass_test_query = """
is_in(51.168825, 7.967821);
out;
"""

def get_polygons(lat=51.485334, lon=6.731369, plant_name=None, api=overpy.Overpass(url='https://z.overpass-api.de/api/interpreter'), delay=1, score_threshold=50):
    def nodes_to_json_compatible(nodes):
        out_list = []
        for el in nodes:
            out_list.append([(float(node.lat), float(node.lon)) for node in el])
        return out_list
    
    def get_relation_nodes(relation):
        ways = [result.get_way(member.ref) for member in relation.members if member.role == 'outer' and isinstance(member, overpy.RelationWay)]
        nodes = [way.get_nodes() for way in ways]
        
        return nodes_to_json_compatible(nodes)
    
    def get_way_nodes(way):
        nodes = [way.get_nodes()]
        
        return nodes_to_json_compatible(nodes)
    
    def get_name(element):
        if 'name' in element.tags:
            return element.tags['name']
        
        return None
    
    def process_result(result, method):
        if len(result.relations) == 1:
            # If there is a single relation, we fetch all outer ways which are part of the relation
            relation = result.relations[0]
            
            return get_relation_nodes(relation), '{} One Relation returned'.format(method), get_name(relation)
        
        elif len(result.relations) > 1: 
            # Disambiguation logic when multiple relations are found -- select relation which most closely resembles plant name
            names = [get_name(relation) for relation in result.relations]
            
            # Any names containing '钢' should force a match
            contains_steel_glyph = [True if isinstance(name, str) and '钢' in name else False for name in names]
            if any(contains_steel_glyph):
                name = names[np.where(contains_steel_glyph)[0][0]]
                score = np.inf
            else:
                name, score = fuzzywuzzy.process.extractOne(plant_name, names)
            relation = result.relations[names.index(name)]

            if name is None:
                score = 0
            if score > score_threshold:
                return (get_relation_nodes(relation),
                        '{} Resolvable ambiguous result comprising several Relations returned ({} {})'.format(method, names, score), get_name(relation))
            
            return (None,
                    '{} Unresolvable ambiguous result comprising several Relations returned ({} {})'.format(method, names, score), None)

        if len(result.ways) == 1:
            # If there is a single way, we fetch it
            way = result.ways[0]

            return get_way_nodes(way), '{} One Way returned'.format(method), get_name(way)

        elif len(result.ways) > 1:
            # Disambiguation logic when multiple ways are found -- select way which most closely resembles plant name
            names = [get_name(way) for way in result.ways]
            
            # Any names containing '钢' should force a match
            contains_steel_glyph = [True if isinstance(name, str) and '钢' in name else False for name in names]
            if any(contains_steel_glyph):
                name = names[np.where(contains_steel_glyph)[0][0]]
                score = np.inf
            else:
                name, score = fuzzywuzzy.process.extractOne(plant_name, names)
            way = result.ways[names.index(name)]

            if name is None:
                score = 0            
            if score > score_threshold:
                return (get_way_nodes(way),
                        '{} Resolvable ambiguous result comprising several Ways returned ({} {})'.format(method, names, score), get_name(way))
            
            return (None,
                    '{} Unresolvable ambiguous result comprising several Ways returned ({} {})'.format(method, names, score), None)

    time.sleep(delay)

    # First, attempt to query based on area membership
    query = overpass_area_membership_query.replace('LAT', str(lat)).replace('LON', str(lon))
    result = api.query(query)
        
    if len(result.relations) > 0 or len(result.ways) > 0:
        return process_result(result, method='Area Query:')
        
    # As a fallback, attempt to query based on proximity
    radius = 16.0
    while radius <= 1024.0:
        time.sleep(delay)

        query = overpass_proximity_query.replace('LAT', str(lat)).replace('LON', str(lon)).replace('RADIUS', str(radius))
        result = api.query(query)
        
        if len(result.relations) > 0 or len(result.ways) > 0:
            return process_result(result, method='Proximity Query:')

        radius *= 2
    
    return None, '{} Empty result comprising no Relations and no Ways at radius {}'.format('Proximity Query:', radius/2), None

def get_polygons_for_df(df):
    for i, row in tqdm(df.iterrows(), total=len(df), desc='Plant queries'):
        if row['osm_overpass_result'] is None or 'traceback' in row['osm_overpass_result'].lower():
            try:
                nodes, result, name = get_polygons(row['latitude'], row['longitude'], row['company_name'])
                df['osm_overpass_polygons'].iloc[i] = nodes
                df['osm_overpass_result'].iloc[i] = result
                df['osm_overpass_name'].iloc[i] = name
            except:
                print('Exception occurred when processing plant {} at lat {} lon {}'.format(row['company_name'], row['latitude'], row['longitude']))
                df['osm_overpass_result'].iloc[i] = traceback.format_exc()
                
        if i % 50 == 0:
            df.to_pickle(CHECKPOINT_FILE)

    return df

# TODO Verify that all polygons start and end with the same node
# TODO Convert node lists to multi-polygons

In [14]:
df_steel_germany['osm_overpass_polygons'] = None
df_steel_germany['osm_overpass_result'] = None
df_steel_germany['osm_overpass_name'] = None

# Attempt to resolve each query up to 10 times:
for i in range(10):
    df_steel_germany = get_polygons_for_df(df_steel_germany)
    
df_steel_germany.to_csv(OUTPUT_FILE_GERMANY)

NameError: name 'df_steel_germany' is not defined

Let us examine the histogram of result status values, as obtained using the heuristic: 

In [20]:
df_steel_germany['osm_overpass_result'].value_counts()

Area Query: One Relation returned    2
Area Query: One Way returned         1
Name: osm_overpass_result, dtype: int64

As observed, the large majority of queries resolve to a single Relation or Way, or are resolvable ambiguous queries. A minority (3) of queries are unresolvable. Let us examine the ambiguous queries:

In [21]:
I = df_steel_germany['osm_overpass_result'].apply(lambda s: 'ambiguous' in s.lower())
df_steel_germany.loc[I, ['company_name', 'location_name', 'latitude', 'longitude', 'osm_overpass_result', 'osm_overpass_name']]

,company_name,location_name,latitude,longitude,osm_overpass_result,osm_overpass_name


In [22]:
for _, row in df_steel_germany.loc[I, 'osm_overpass_result'].iteritems():
    print(row)

As observed, 'ENERGIETECHNIK ESSEN GMBH' appears to correspond to two relatively small secondary industrial facilities. This location would likely not be discernable as a steel-related plant from satellite imagery. On the other hand, 'DK RECYCL.+ ROHEISEN' is clearly a steel-related facility, discernable from satellite imagery. 

Let us explore queries which resolve to non-null names:

In [23]:
I = df_steel_germany['osm_overpass_name'].notnull()
df_steel_germany.loc[I, ['company_name', 'location_name', 'latitude', 'longitude', 'osm_overpass_result', 'osm_overpass_name']]

,company_name,location_name,latitude,longitude,osm_overpass_result,osm_overpass_name
0,THYSSENKRUPP STEEL EUROPE AG,DUISBURG-HAMBORN,51.485334,6.731369,Area Query: One Relation returned,ThyssenKrupp Steel Europe AG
1,THYSSENKRUPP STEEL EUROPE AG,DUISBURG-BRUCKHAUSEN,51.493014,6.740669,Area Query: One Relation returned,ThyssenKrupp Steel Europe AG
2,THYSSENKRUPP STEEL EUROPE AG,DUISBURG-SCHWELGERN,51.506862,6.739497,Area Query: One Way returned,ThyssenKrupp Steel Europe AG


Based on cursory exploration, the majority of queries appear to resolve to names which suggest steel-related activity. Exceptions are 'VULKAN INOX GMBH' and 'C.D. WÌãLZHOLZ GMBH', which both appear to resolve to surrounding industrial areas, while being small facilities themselves.

Attempt to extract polygons for the entire dataset:

In [12]:
df_steel.head()

,country,location_id,company_name,number_plants,number_blast_furnace,total_capacity,location_name,latitude,longitude,accuracy,uid,geometry
0,GERMANY,100021,THYSSENKRUPP STEEL EUROPE AG,2,2.0,3700.0,DUISBURG-HAMBORN,51.485334,6.731369,Exact,DEU0001,POINT (6.73137 51.48533)
1,GERMANY,100022,THYSSENKRUPP STEEL EUROPE AG,16,0.0,27544.0,DUISBURG-BRUCKHAUSEN,51.493014,6.740669,Exact,DEU0002,POINT (6.74067 51.49301)
2,GERMANY,100023,THYSSENKRUPP STEEL EUROPE AG,6,2.0,22815.0,DUISBURG-SCHWELGERN,51.506862,6.739497,Exact,DEU0003,POINT (6.73950 51.50686)
3,GERMANY,100024,THYSSENKRUPP STEEL EUROPE AG,16,0.0,29280.0,DUISBURG-BEECKERWERTH,51.483780,6.706625,Exact,DEU0004,POINT (6.70662 51.48378)
4,GERMANY,100025,THYSSENKRUPP STEEL EUROPE AG,3,0.0,1620.0,DUISBURG-HÌÏTTENHEIM,51.365948,6.710592,Exact,DEU0005,POINT (6.71059 51.36595)


In [12]:
df_steel['osm_overpass_polygons'] = None
df_steel['osm_overpass_result'] = None
df_steel['osm_overpass_name'] = None

In [10]:
# Attempt to resolve each query up to 10 times:
for i in range(10):
    df_steel = get_polygons_for_df(df_steel)
    
df_steel.to_csv(OUTPUT_FILE)
df_steel.to_pickle(CHECKPOINT_FILE)

Plant queries:  44%|████▍     | 901/2054 [00:19<00:04, 232.20it/s]/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
Plant queries:  72%|███████▏  | 1481/2054 [42:54<5:37:43, 35.36s/it]

Exception occurred when processing plant LENGSHUIJIANG I&S at lat 27.686251000000002 lon 111.43498400000001


Plant queries:  72%|███████▏  | 1482/2054 [44:10<7:33:54, 47.61s/it]

Exception occurred when processing plant TIANFENG I&S CO at lat 38.947511999999996 lon 116.97413


Plant queries:  72%|███████▏  | 1483/2054 [44:49<7:06:31, 44.82s/it]

...

Plant queries:  72%|███████▏  | 1486/2054 [45:54<4:30:08, 28.54s/it]

Exception occurred when processing plant TIANJIN JUDA STEEL IND. CO, LTD at lat 39.343357399999995 lon 117.3616476


Plant queries:  73%|███████▎  | 1505/2054 [53:47<3:16:41, 21.50s/it]

Exception occurred when processing plant Yichang Three Gorges Quantong Co., Ltd. at lat 30.691965999999997 lon 111.28647099999999


Plant queries: 100%|██████████| 2054/2054 [00:07<00:00, 287.69it/s]


In [9]:
df_steel = pd.read_pickle(CHECKPOINT_FILE)